# Surrogate Model Training Notebook
This notebook trains surrogate models to predict **Cost** from the ABM input parameters.
Datasets available: `200runs_complete.csv`, `400runs_complete.csv`, `800runs_complete.csv`, `1600runs_complete.csv`.


In [ ]:
import os
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt
import joblib

RANDOM_STATE = 42


## Decision framework (ABM vs Surrogate Model)
The framework starts by deciding **in advance** whether it is worth building a surrogate model (SM) instead of running the ABM for all planned evaluations.




In [ ]:
N_total = 1600

train_fracs = [0.125, 0.25, 0.50]
n_trains = [int(round(N_total * f)) for f in train_fracs]

df_budget = pd.DataFrame({
    "train_frac": train_fracs,
    "n_train": n_trains,
})
df_budget["R"] = df_budget["n_train"] / N_total
df_budget["SMBI"] = 1 - df_budget["R"]

print("Training budgets to test (evolution study):")
df_budget


Training budgets to test (evolution study):


,train_frac,n_train,R,SMBI
0,0.125,200,0.125,0.875
1,0.250,400,0.250,0.750
2,0.500,800,0.500,0.500


## 1) Load a dataset


In [ ]:
DATA_DIR = "."
available = sorted(glob.glob(os.path.join(DATA_DIR, "*runs_complete.csv")))
available


['.\\1600runs_complete.csv',
 '.\\200runs_complete.csv',
 '.\\400runs_complete.csv',
 '.\\800runs_complete.csv']

In [ ]:
# --- Data loading ---

DATA_DIR = "."
FULL_DATA_PATH = "1600runs_complete.csv" 

def try_load_budget_file(n_train: int):
    patterns = [
        f"{n_train}runs_complete.csv",
        f"{n_train}runs.csv",
        f"*{n_train}*runs*.csv",
    ]
    for pat in patterns:
        matches = glob.glob(os.path.join(DATA_DIR, pat))
        if matches:
            # pick the first match deterministically
            return pd.read_csv(matches[0]), matches[0]
    return None, None

def load_full_dataset():
    full_path = os.path.join(DATA_DIR, FULL_DATA_PATH)
    if os.path.exists(full_path):
        return pd.read_csv(full_path), full_path
    matches = glob.glob(os.path.join(DATA_DIR, "*1600*runs*.csv"))
    if matches:
        return pd.read_csv(matches[0]), matches[0]
    raise FileNotFoundError(
        "Could not find a full dataset. Please put a 1600-run CSV in DATA_DIR "
        "and set FULL_DATA_PATH correctly."
    )

df_full, full_used = load_full_dataset()
print(f"Loaded full dataset: {full_used}  shape={df_full.shape}")
df_full.head()


Loaded full dataset: .\1600runs_complete.csv  shape=(1600, 11)


,ageingParents,moveInWithKids,retiredHours,ageOfRetirement,personCareProb,maleAgeCareScaling,femaleAgeCareScaling,childHours,homeAdultHours,workingAdultHours,Cost
0,0.10000,0.000200,40.000,55.0000,0.000200,10.00000,10.00000,1.0000,5.00000,5.00000,40362.71766
1,0.14500,0.000295,44.500,57.4500,0.000295,11.87500,11.87500,1.7500,9.37500,8.12500,16286.27817
2,0.12250,0.000342,42.250,58.6750,0.000248,12.81250,10.93750,2.1250,11.56250,6.56250,14776.62568
3,0.16750,0.000248,46.750,56.2250,0.000342,10.93750,12.81250,1.3750,7.18750,9.68750,17173.13121
4,0.11125,0.000319,47.875,59.2875,0.000319,10.46875,11.40625,1.5625,12.65625,8.90625,21350.79183


In [27]:
df_full.shape, df_full.dtypes


((1600, 11),
 ageingParents           float64
 moveInWithKids          float64
 retiredHours            float64
 ageOfRetirement         float64
 personCareProb          float64
 maleAgeCareScaling      float64
 femaleAgeCareScaling    float64
 childHours              float64
 homeAdultHours          float64
 workingAdultHours       float64
 Cost                    float64
 dtype: object)

## 2) Basic sanity checks

In [28]:
# Missing values?
df_full.isna().sum()


ageingParents           0
moveInWithKids          0
retiredHours            0
ageOfRetirement         0
personCareProb          0
maleAgeCareScaling      0
femaleAgeCareScaling    0
childHours              0
homeAdultHours          0
workingAdultHours       0
Cost                    0
dtype: int64

In [29]:
# Summary stats
df_full.describe().T


,count,mean,std,min,25%,50%,75%,max
ageingParents,1600.0,0.408266,0.214907,0.100000,0.197412,0.394629,0.591943,0.799219
moveInWithKids,1600.0,0.000797,0.000442,0.000200,0.000398,0.000695,0.001199,0.001599
retiredHours,1600.0,59.731445,11.733234,40.000000,49.741211,59.482422,69.970703,79.960938
ageOfRetirement,1600.0,64.959807,5.786178,55.000000,59.972608,64.945215,69.917822,74.990234
personCareProb,1600.0,0.000823,0.000429,0.000200,0.000397,0.000799,0.001198,0.001598
maleAgeCareScaling,1600.0,17.493848,4.331460,10.000000,13.746338,17.492676,21.239014,24.985352
femaleAgeCareScaling,1600.0,17.492676,4.331456,10.000000,13.746338,17.492676,21.239014,24.985352
childHours,1600.0,5.122012,2.713246,1.000000,2.498535,4.995117,7.492675,9.990234
homeAdultHours,1600.0,20.469800,10.788938,5.000000,13.745728,18.116455,22.487182,49.946289
workingAdultHours,1600.0,16.403833,8.384943,5.000000,11.246948,14.368896,17.490845,39.956055


## 3) Build Surrogate models

In [ ]:
def evaluate(model, X_train, y_train, X_test, y_test, cv_folds=5):
    # Fit
    model.fit(X_train, y_train)
    # Predict
    y_pred = model.predict(X_test)
    # Metrics
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Cross-validated RMSE (on training set)
    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)
    cv_rmse = -cross_val_score(
        model, X_train, y_train,
        scoring="neg_root_mean_squared_error",
        cv=cv,
        n_jobs=-1
    )
    return {
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2,
        "rmse_cv_mean": float(np.mean(cv_rmse)),
        "rmse_cv_std": float(np.std(cv_rmse)),
    }

TARGET = "Cost"

def make_xy(df: pd.DataFrame, target: str):
    X = df.drop(columns=[target])
    y = df[target].astype(float)
    X = X.astype(float)
    return X, y

def get_training_dataframe(n_train: int, random_state: int = RANDOM_STATE):
    df_budget, used_path = try_load_budget_file(n_train)
    if df_budget is not None:
        print(f"Using budget-specific dataset for n_train={n_train}: {used_path}  shape={df_budget.shape}")
        return df_budget, used_path

    if n_train > len(df_full):
        raise ValueError(f"Requested n_train={n_train}, but full dataset has only {len(df_full)} rows.")
    df_sample = df_full.sample(n=n_train, random_state=random_state).reset_index(drop=True)
    print(f"Sampled n_train={n_train} rows from full dataset ({full_used}).")
    return df_sample, f"SAMPLED_FROM:{os.path.basename(full_used)}"

numeric_preprocess = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Models to compare
models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=100,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),
    
    'Gradient Boosted Trees': GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=RANDOM_STATE
    ),
    
    'Decision Tree': DecisionTreeRegressor(
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=RANDOM_STATE
        )
       
}

all_results = []
best_models = {}

for n_train in n_trains:
    R = n_train / N_total
    SMBI = 1 - R

    df_train, source_used = get_training_dataframe(n_train)
    X, y = make_xy(df_train, TARGET)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE
    )

    # Wrap each model in a pipeline
    local_results = []
    local_fitted = {}

    for name, base_model in models.items():
        pipe = Pipeline(steps=[
            ("preprocess", numeric_preprocess),
            ("model", base_model)
        ])
        metrics = evaluate(pipe, X_train, y_train, X_test, y_test, cv_folds=5)

        row = {
            "n_train": n_train,
            "train_frac": R,
            "SMBI": SMBI,
            "data_source": source_used,
            "model": name,
            **metrics
        }
        local_results.append(row)
        local_fitted[name] = pipe

    local_df = pd.DataFrame(local_results).sort_values(by="RMSE")
    # pick best by RMSE (you can change to -R2 if preferred)
    best_name = local_df.iloc[0]["model"]
    best_models[n_train] = (best_name, local_fitted[best_name])

    all_results.append(local_df)

results_long = pd.concat(all_results, ignore_index=True)
results_long.sort_values(["n_train", "RMSE"]).head(10)


Using budget-specific dataset for n_train=200: .\200runs_complete.csv  shape=(200, 11)


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean s

Using budget-specific dataset for n_train=400: .\400runs_complete.csv  shape=(400, 11)


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean s

Using budget-specific dataset for n_train=800: .\800runs_complete.csv  shape=(800, 11)


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean s

,n_train,train_frac,SMBI,data_source,model,RMSE,MAE,R2,rmse_cv_mean,rmse_cv_std
0,200,0.125,0.875,.\200runs_complete.csv,RandomForest,2322.484345,1836.066268,0.915586,3163.154465,727.504077
1,200,0.125,0.875,.\200runs_complete.csv,Gradient Boosted Trees,2828.840824,2157.375827,0.874765,3370.726395,1131.852804
2,200,0.125,0.875,.\200runs_complete.csv,Decision Tree,6394.065970,3996.236811,0.360170,4863.473256,534.414682
3,400,0.250,0.750,.\400runs_complete.csv,Gradient Boosted Trees,3199.150779,1606.559182,0.857326,1989.127791,211.474305
4,400,0.250,0.750,.\400runs_complete.csv,RandomForest,3448.069536,1876.137642,0.834260,2277.899896,125.340459
5,400,0.250,0.750,.\400runs_complete.csv,Decision Tree,5103.772200,2759.281343,0.636874,3384.724146,375.277552
6,800,0.500,0.500,.\800runs_complete.csv,Gradient Boosted Trees,980.436398,701.498314,0.976133,1327.398626,433.753172
7,800,0.500,0.500,.\800runs_complete.csv,RandomForest,1235.348073,890.315847,0.962110,1509.331156,401.899898
8,800,0.500,0.500,.\800runs_complete.csv,Decision Tree,2180.134449,1513.265676,0.881990,2316.376241,169.000215
